In [312]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import validators
import datetime
import math
import re
import ipynb
import warnings
import time

In [313]:
warnings.filterwarnings("ignore")

In [314]:
def find_tables_qb(first, last, team, year):
    for i in range(0,10):
        try:
            if len(last) < 4:
                last = last + 'x'
            url = 'https://www.pro-football-reference.com/players/%s/%s%s0%i.htm' % (last[0], last[0:4], first[0:2], i)
            if first == 'Derek' and last == 'Carr':
                url = 'https://www.pro-football-reference.com/players/C/CarrDe02.htm'
            elif last == 'Mariota':
                url = 'https://www.pro-football-reference.com/players/M/MariMa01.htm'
            response = requests.get(url)
            if response.status_code != 404:
                x = pd.read_html(url)
                df = x[0]
                df[df.columns[0]] = df[df.columns[0]].str.replace(r'[\*+]', '', regex=True)
                if (df[df[df.columns[0]] == str(year)][df.columns[2]]).values == team:
                    for i in range(1, len(x)):
                        df2 = x[i]
                        if ('Rushing', 'Yds') in df2.columns:
                            df2[df2.columns[0]] = df2[df2.columns[0]].str.replace(r'[\*+]', '', regex=True)
                            return x[0], x[i], response
        except:
            pass
    return 0, 0, 0 

def find_age(response, year):
    soup = BeautifulSoup(response.content, "html.parser")
    x = soup.find('div', {'id': 'info'})
    dob = x.find('span"', {'id': 'necro-birth'})['data-birth']
    return math.floor(((datetime.date(year, 9, 7) - datetime.date(int(dob[0:4]), int(dob[5:7]), int(dob[8:10]))).days)/365)

In [315]:
qbs = pd.read_csv('qbs.csv')

In [316]:
qbs.drop(columns = 'Unnamed: 0', inplace = True)

In [317]:
qb2022 = pd.read_csv('2022qbs.csv')
corr_teams = pd.read_html('https://www.pro-football-reference.com/years/2022/fantasy.htm')
corrqb2022 = corr_teams[0][[('Unnamed: 1_level_0',  'Player'), ('Unnamed: 2_level_0',      'Tm'), ('Unnamed: 3_level_0', 'FantPos')]]

In [318]:
corrqb2022[('Unnamed: 1_level_0',  'Player')] = corrqb2022[('Unnamed: 1_level_0',  'Player')].str.replace(r'[\*+]', '', 
                                                                                                regex=True)

In [319]:
corrqb2022 = corrqb2022[corrqb2022[('Unnamed: 3_level_0', 'FantPos')] == 'QB']

In [320]:
# qb2022 = qb2022.nlargest(50, 'FPTS')
qb2022['Player'] = qb2022['Player'].replace('III', '', regex = True)
qb2022['Player'] = qb2022['Player'].replace('II', '', regex = True)

In [321]:
x = qb2022['Player'].str.split(' ', expand = True)

In [322]:
for i in range(0, len(x)):
    if x.loc[i,2] == "":
        x.loc[i,2] = x.loc[i,3]
x.drop(columns = 3, inplace = True)
x[2] = x[2].replace(r'[()]', '', regex = True)

In [323]:
qb2022[['First', 'Last', 'Tm']] = x
qb2022['Player'] = [i.strip() if isinstance(i,str) else i for i in qb2022['Player'].str.split('(', expand = True)[0]]
qb2022['Year'] = 2022
qb2022['Pos'] = 'QB'
qb2022.dropna(inplace = True)
qb2022['Age'] = None
qb2022.drop(columns = 'Rank', inplace = True)

In [324]:
x = pd.DataFrame()
x['Player'] = corrqb2022[('Unnamed: 1_level_0',  'Player')]
x['Tm'] = corrqb2022[('Unnamed: 2_level_0',      'Tm')]
x['Pos'] = corrqb2022[('Unnamed: 3_level_0', 'FantPos')]

In [325]:
qb2022 = pd.merge(qb2022, x, on = 'Player')

In [326]:
qb2022.drop(columns = ['Tm_x', 'Pos_x'], inplace = True)
qb2022.rename(columns = {'Tm_y': 'Tm', 'Pos_y': 'Pos'}, inplace = True)

In [328]:
column_order = ['First', 'Last', 'Year', 'Tm', 'Pos', 'Age', 'FPTS', 'Player']
qb2022 = qb2022.reindex(columns = column_order)
qb2022.rename(columns = {'FPTS': 'FantasyPoints'}, inplace = True)

In [329]:
totalqbs = pd.concat([qbs, qb2022])
totalqbs = totalqbs.reset_index().drop(columns = 'index')

In [330]:
totalqbs = totalqbs.sort_values('FantasyPoints', ascending = False).groupby('Year').head(40)
totalqbs = totalqbs.reset_index().drop(columns = 'index')

In [332]:
totalqbs[totalqbs['Tm'] == '2TM']

,First,Last,Year,Tm,Pos,Age,FantasyPoints,Player
489,Baker,Mayfield,2022,2TM,QB,NaN,129.40,Baker Mayfield
600,Kyle,Orton,2011,2TM,QB,29.0,85.62,Kyle Orton
646,Ryan,Mallett,2015,2TM,QB,27.0,68.94,Ryan Mallett
652,Brandon,Weeden,2015,2TM,QB,32.0,66.42,Brandon Weeden
657,Matt,Cassel,2015,2TM,QB,33.0,64.84,Matt Cassel
661,Brian,Hoyer,2017,2TM,QB,32.0,63.88,Brian Hoyer


In [333]:
totalqbs['Last'] = totalqbs['Last'].replace("'", '', regex = True)

In [334]:
totalqbs['QBR'] = np.nan
totalqbs['PassAtt'] = np.nan
totalqbs['PassYds'] = np.nan
totalqbs['PassTD'] = np.nan
totalqbs['PassCmp'] = np.nan
totalqbs['Int'] = np.nan
totalqbs['RushAtt'] = np.nan
totalqbs['RushYards'] = np.nan
totalqbs['RushTD'] = np.nan
totalqbs['Fumbles'] = np.nan
totalqbs['Experience'] = np.nan

In [2]:
start = time.time()
for i in range(len(totalqbs)):
#     if i % 100 == 0:
#         print(i, totalqbs.loc[i, 'First'], totalqbs.loc[i, 'Last'])
    try:    
        passing, rushing, response = find_tables_qb(totalqbs.loc[i, 'First'],totalqbs.loc[i, 'Last'],
                                                    totalqbs.loc[i, 'Tm'], totalqbs.loc[i, 'Year'])
        
        passing['Year'] = pd.to_numeric(passing['Year'], errors = 'coerce')
        rushing[(( 'Unnamed: 0_level_0',  'Year'))] = pd.to_numeric(rushing[(( 'Unnamed: 0_level_0',  'Year'))], 
                                                                    errors = 'coerce')
        
        if totalqbs.loc[i, 'Tm'] == '2TM':
            idx = passing[(passing['Tm'] == '2TM') & (passing['Year'] == totalqbs.loc[i, 'Year'])].index + 1
            totalqbs.loc[i, 'Tm'] = passing.loc[idx, 'Tm'].iloc[0]
        
        passing.dropna(subset = ['Age'], inplace = True)
        rushing.dropna(subset = [(( 'Unnamed: 1_level_0',   'Age'))], inplace = True)
        
        totalqbs.loc[i, 'Experience'] = len(passing[passing['Year'] <= totalqbs.loc[i, 'Year']])
        
        if totalqbs.loc[i, 'Year'] == 2022:
            totalqbs.loc[i, 'Age'] = find_age(response, totalqbs.loc[i, 'Year'])
        
        date = passing[(passing['Year'] < (totalqbs.loc[i, 'Year'])) & (passing['Year'] >= (totalqbs.loc[i, 'Year'] - 2))].reset_index()
        date.drop(columns = ['Tm', 'Pos', 'QBrec'], inplace = True)
        if 'Awards' in date.columns:
            date.drop(columns = ['Awards'], inplace = True)
        date = date.astype(float)
        
        totalqbs.loc[i, 'QBR'] = (date['QBR']*date['G']*(date.index+1)).sum()/(date['G']*(date.index+1)).sum()
        totalqbs.loc[i, 'PassTD'] = (date['TD']*date['G']*(date.index+1)).sum()/(date['G']*(date.index+1)).sum()
        totalqbs.loc[i, 'Int'] = (date['Int']*date['G']*(date.index+1)).sum()/(date['G']*(date.index+1)).sum()
        totalqbs.loc[i, 'PassCmp'] = (date['Cmp']*date['G']*(date.index+1)).sum()/(date['G']*(date.index+1)).sum()
        totalqbs.loc[i, 'PassAtt'] = (date['Att']*date['G']*(date.index+1)).sum()/(date['G']*(date.index+1)).sum()
        totalqbs.loc[i, 'PassYds'] = (date['Yds']*date['G']*(date.index+1)).sum()/(date['G']*(date.index+1)).sum()
        
        date = rushing[(rushing[(( 'Unnamed: 0_level_0',  'Year'))] < (totalqbs.loc[i, 'Year'])) & (rushing[(( 'Unnamed: 0_level_0',  'Year'))] >= (totalqbs.loc[i, 'Year'] - 2))].reset_index()
        date[('Receiving', 'Ctch%')] = date[('Receiving', 'Ctch%')].replace('%', '', regex = True)
        date.drop(columns = [( 'Unnamed: 2_level_0',     'Tm'), 
                             ( 'Unnamed: 3_level_0',     'Pos')], inplace = True)
        if ('Unnamed: 32_level_0', 'Awards') in date.columns:
            date.drop(columns = [('Unnamed: 32_level_0', 'Awards')], inplace = True)
        date = date.astype(float)
        
        totalqbs.loc[i, 'RushAtt'] = (date[('Rushing', 'Att')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalqbs.loc[i, 'RushYards'] = (date[('Rushing', 'Yds')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalqbs.loc[i, 'RushTD'] = (date[('Rushing', 'TD')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games','G')]*(date.index+1)).sum()
        totalqbs.loc[i, 'Fumbles'] = (date[('Unnamed: 30_level_0', 'Fmb')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
    except:
        pass
#         print("Error:", totalqbs.loc[i, 'First'], totalqbs.loc[i, 'Last'], 
#               totalqbs.loc[i, 'Tm'], totalqbs.loc[i, 'Year'])
    time.sleep(10)
print(time.time() - start)

NameError: name 'time' is not defined

In [345]:
finalqbs = totalqbs.dropna()

In [347]:
finalqbs.to_csv('finalqbs.csv')